In [1]:
import mysql.connector
import pandas as pd
import jaconv
from tqdm import tqdm
import os
from concurrent.futures import ProcessPoolExecutor

from utils.connect_cookpad_db import *

In [2]:
config = get_config("../config.ini")
OUTPUT_ROOT = config.get("Output","output_path")

グラフのもととなるCSVの作成

In [6]:
sql = "SELECT * FROM recipes;"  # あなたのSQLクエリをここに入れます
cookpad_recipe_data = execute_sql2df(config, sql)
sql = "SELECT * FROM ingredients;"
cookpad_ingredient_data  = execute_sql2df(config, sql)

/home/yoshimaru/Japanese-FoodKG-master/src/utils/connect_cookpad_db.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


In [3]:
sql = """
CREATE INDEX recipe_id_index ON ingredients(recipe_id);
"""
marge_cookpad_data = execute_sql2df(config, sql)

/home/yoshimaru/Japanese-FoodKG-master/src/utils/connect_cookpad_db.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


TypeError: 'NoneType' object is not iterable

In [4]:
sql = """
SELECT 
    recipes.id AS recipe_id,
    recipes.title AS recipe_title,
    recipes.description AS recipe_description,
    ingredients.name AS ingredient_name,
    ingredients.quantity AS ingredient_quantity
FROM 
    recipes
INNER JOIN 
    ingredients ON recipes.id = ingredients.recipe_id;

"""
marge_cookpad_data = execute_sql2df(config, sql)

In [9]:
marge_cookpad_data.dropna(subset=['ingredient_name'], inplace=True)
marge_cookpad_data["ingredient_name"] = [jaconv.kata2hira(name) for name in marge_cookpad_data["ingredient_name"]] 

In [11]:
marge_cookpad_data.dropna(subset=['recipe_title'], inplace=True)
marge_cookpad_data["recipe_title"] = [jaconv.kata2hira(name) for name in marge_cookpad_data["recipe_title"]] 

In [16]:
marge_cookpad_data["data_source"] = "cookpad"
marge_cookpad_data["edge_type"] = "recipe-ingredient"
cookpad_edge = marge_cookpad_data[["recipe_id","recipe_title", "ingredient_name","edge_type","data_source"]].copy()

In [18]:
save_path = os.path.join(OUTPUT_ROOT,"output_csv","cookpad_edges.csv") 
cookpad_edge.to_csv(save_path,index=False)